# Apache Hudi Core Conceptions (7) - COW: Bucket Index

## 1. Configuration

In [1]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

In [2]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar
# deploy hudi-stat.sh - a utility shell script 
wget https://github.com/bluishglc/hudi-core-conceptions/releases/download/v1.0/hudi-stat.sh -O ~/hudi-stat.sh &>/dev/null
chmod a+x ~/hudi-stat.sh
ls ~/hudi-stat.sh

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-07 05:55 /tmp/hudi-spark-bundle.jar
/home/emr-notebook/hudi-stat.sh


In [3]:
%%html
<style>
table {float:left}
</style>

## 2. COW + BUCKET INDEX (SIMPLE HASHING)

Note：for SIMPLE BUCKET INDEX， there is no limit to the size of files (<=120MB), in other words, The `hoodie.parquet.max.file.size because` and `hoodie.parquet.small.file.limit` do NOT work under SIMPLE BUCKET INDEX, because the buckets number is fixed, so file groups number is fixed, it is NOT allowed to split any new file groups, otherwise, the mapping ralationship between record key and fileId will be broken.

KEY|VALUE
:---|:---
hoodie.index.type|BUCKET
hoodie.copyonwrite.record.size.estimate|219

### 2.1. Create Table

In [4]:
%%sql
set TABLE_NAME=reviews_cow_bucket_1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
15,application_1678096020253_0035,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env TABLE_NAME=reviews_cow_bucket_1

env: TABLE_NAME=reviews_cow_bucket_1


In [6]:
%%sql
set TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_cow_bucket_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [7]:
%env TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_cow_bucket_1

env: TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_cow_bucket_1


In [8]:
%%sh
echo $(basename $TABLE_PATH)
aws s3 rm $TABLE_PATH --recursive &>/dev/null
rm -rf ~/${TABLE_NAME}
sleep 3

reviews_cow_bucket_1


In [9]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [10]:
%%sql
create table if not exists ${TABLE_NAME}(
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location '${TABLE_PATH}'
partitioned by (parity)
options ( 
    type = 'cow',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.index.type = 'BUCKET',
    hoodie.bucket.index.num.buckets = '2'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.2. Batch 1 - Insert ( 104 MB / FileGroup )

In [11]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2007

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sh
~/hudi-stat.sh $TABLE_PATH timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230307055608090 │ commit │ COMPLETED │ 03-07 05:56 │ 03-07 05:56 │ 03-07 05:56 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪═════════════════════╪════════════════

The 1st batch insert has exceeded `hoodie.parquet.small.file.limit` (100MB), let's see what will hadppend next.

### 2.3. Batch 2 - Insert ( 129 MB / FileGroup )

In [13]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 2008

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sh
~/hudi-stat.sh $TABLE_PATH timeline commits storage


[ TIMELINE ]

╔═════╤═══════════════════╤════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │        │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230307055608090 │ commit │ COMPLETED │ 03-07 05:56 │ 03-07 05:56 │ 03-07 05:56 ║
╟─────┼───────────────────┼────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230307055723252 │ commit │ COMPLETED │ 03-07 05:57 │ 03-07 05:57 │ 03-07 05:58 ║
╚═════╧═══════════════════╧════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Adde

Evidence: latest version of parquet file is over `hoodie.parquet.small.file.limit` ( 120 MB ), no new file group is splited.
Conclusion: the default file sizing rule [100MB, 120MB] is inapplicable 